In [1]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import datetime as dt
import pandas as pd
import xarray as xr
import cartopy.crs as ccrs
import matplotlib.cm as cm
from matplotlib import ticker, cm


import cartopy.feature as cfeature
import cartopy.mpl.ticker as cticker
from cartopy.util import add_cyclic_point
from mpl_toolkits.basemap import Basemap
import matplotlib.colors as colors
import matplotlib.cbook as cbook
import matplotlib.ticker as ticker
from ipywidgets import interactive

In [2]:
os.chdir("/scratch/m/m300814/cesm/output/f.e22.FHIST.f09_f09_mg17.MP/run")

NameError: name 'os' is not defined

In [3]:
emission_pnas=xr.open_dataset('/work/mh0735/m300814/new_emissions/plastics_bin_source.nc',decode_times=False)

In [4]:
emission_pnas=emission_pnas['emiss']

In [5]:
emission_pnas

<xarray.DataArray 'emiss' (bin: 6, nvars: 5, lat: 192, lon: 288)>
[1658880 values with dtype=float32]
Coordinates:
  * bin      (bin) int32 1 2 3 4 5 6
  * nvars    (nvars) int32 1 2 3 4 5
  * lat      (lat) float32 -90.0 -89.06 -88.12 -87.17 ... 87.17 88.12 89.06 90.0
  * lon      (lon) float32 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
Attributes:
    units:    kg/m2/s

In [6]:
def area_grid(lat, lon):
    """
    Calculate the area of each grid cell
    Area is in square meters
    
    Input
    -----------
    lat: vector of latitude in degrees
    lon: vector of longitude in degrees
    
    Output
    -----------
    area: grid-cell area in square-meters with dimensions, [lat,lon]
    
    Notes
    -----------
    Based on the function in
    https://github.com/chadagreene/CDT/blob/master/cdt/cdtarea.m
    """
    from numpy import meshgrid, deg2rad, gradient, cos
    from xarray import DataArray

    xlon, ylat = meshgrid(lon, lat)
    R = 6371000

    dlat = deg2rad(gradient(ylat, axis=0))
    dlon = deg2rad(gradient(xlon, axis=1))

    dy = dlat * R
    dx = dlon * R * cos(deg2rad(ylat))
    

    area = dy * dx
    


    xda = DataArray(
        area,
        dims=["lat", "lon"],
        coords={"lat": lat, "lon": lon},
        attrs={
        "long_name": "area_per_pixel",
        "description": "area per pixel",
        "units": "m^2",
        },
    )
    return xda

area = area_grid(emission_pnas['lat'],emission_pnas['lon'])

#area.to_netcdf('area.nc')


In [7]:
ocean=emission_pnas[:,0,:,:]
pop=emission_pnas[:,1,:,:]
dustpop=emission_pnas[:,2,:]
dustag=emission_pnas[:,3:,:]
road=emission_pnas[:,4:,:]

In [8]:
sum_ocean_t=(ocean[0,:,:]*area+ocean[1,:,:]*area+ocean[2,:,:]*area+ocean[3,:,:]*area+ocean[4,:,:]*area+ocean[5,:,:]*area).sum()*3600*24*365*1000
#g/yr

In [9]:
sum_ocean=(ocean[0,:,:]*area,ocean[1,:,:]*area,ocean[2,:,:]*area,ocean[3,:,:]*area,ocean[4,:,:]*area,ocean[5,:,:]*area)

In [10]:
for i in range(6):
    display(sum_ocean[i].sum()*3600*24*365*1000)

<xarray.DataArray ()>
array(32829.81583932)
Coordinates:
    bin      int32 1
    nvars    int32 1

<xarray.DataArray ()>
array(351355.62177889)
Coordinates:
    bin      int32 2
    nvars    int32 1

<xarray.DataArray ()>
array(2894774.50092548)
Coordinates:
    bin      int32 3
    nvars    int32 1

<xarray.DataArray ()>
array(9841612.94096936)
Coordinates:
    bin      int32 4
    nvars    int32 1

<xarray.DataArray ()>
array(1.09065569e+08)
Coordinates:
    bin      int32 5
    nvars    int32 1

<xarray.DataArray ()>
array(7.34599936e+08)
Coordinates:
    bin      int32 6
    nvars    int32 1

In [11]:
sum_road_t=(road[0,:,:]*area+road[1,:,:]*area+road[2,:,:]*area+road[3,:,:]*area+road[4,:,:]*area+road[5,:,:]*area).sum()*3600*24*365*1000

In [12]:
sum_road=(road[0,:,:]*area,road[1,:,:]*area,road[2,:,:]*area,road[3,:,:]*area,road[4,:,:]*area,road[5,:,:]*area)

In [13]:
for i in range(6):
    display(sum_road[i].sum()*3600*24*365*1000)

<xarray.DataArray ()>
array(6.97795513e+09)
Coordinates:
    bin      int32 1

<xarray.DataArray ()>
array(6.69002103e+10)
Coordinates:
    bin      int32 2

<xarray.DataArray ()>
array(2.01631027e+11)
Coordinates:
    bin      int32 3

<xarray.DataArray ()>
array(2.36368889e+12)
Coordinates:
    bin      int32 4

<xarray.DataArray ()>
array(7.00440286e+12)
Coordinates:
    bin      int32 5

<xarray.DataArray ()>
array(1.29809095e+13)
Coordinates:
    bin      int32 6

In [14]:
sum_dustag_t=(dustag[0,:,:]*area+dustag[1,:,:]*area+dustag[2,:,:]*area+dustag[3,:,:]*area+dustag[4,:,:]*area+dustag[5,:,:]*area).sum()*3600*24*365*1000

In [15]:
sum_dustag=(dustag[0,:,:]*area,dustag[1,:,:]*area,dustag[2,:,:]*area,dustag[3,:,:]*area,dustag[4,:,:]*area,dustag[5,:,:]*area)

In [16]:
sum_dustag[5].sum()*3600*24*365*1000

<xarray.DataArray ()>
array(1.30011429e+13)
Coordinates:
    bin      int32 6

In [17]:
sum_dustpop_t=(dustpop[0,:,:]*area+dustpop[1,:,:]*area+dustpop[2,:,:]*area+dustpop[3,:,:]*area+dustpop[4,:,:]*area+dustpop[5,:,:]*area).sum()*3600*24*365*1000

In [18]:
sum_dustpop=(dustpop[0,:,:]*area,dustpop[1,:,:]*area,dustpop[2,:,:]*area,dustpop[3,:,:]*area,dustpop[4,:,:]*area,dustpop[5,:,:]*area)

In [19]:
for i in range(6):
    display(sum_dustpop[i].sum()*3600*24*365*1000)

<xarray.DataArray ()>
array(2523880.96417331)
Coordinates:
    bin      int32 1
    nvars    int32 3

<xarray.DataArray ()>
array(8.20821687e+08)
Coordinates:
    bin      int32 2
    nvars    int32 3

<xarray.DataArray ()>
array(3.10349456e+09)
Coordinates:
    bin      int32 3
    nvars    int32 3

<xarray.DataArray ()>
array(3.6600141e+09)
Coordinates:
    bin      int32 4
    nvars    int32 3

<xarray.DataArray ()>
array(4.96733863e+09)
Coordinates:
    bin      int32 5
    nvars    int32 3

<xarray.DataArray ()>
array(5.74731855e+09)
Coordinates:
    bin      int32 6
    nvars    int32 3

In [25]:
sum_pop_t=(pop[0,:,:]*area+pop[1,:,:]*area+pop[2,:,:]*area+dustpop[3,:,:]*area+pop[4,:,:]*area+pop[5,:,:]*area).sum()*3600*24*365*1000

In [26]:
sum_pop=(pop[0,:,:]*area,pop[1,:,:]*area,pop[2,:,:]*area,pop[3,:,:]*area,dustpop[4,:,:]*area,pop[5,:,:]*area)

In [27]:
for i in range(6):
    display(sum_pop[i].sum()*3600*24*365*1000)

<xarray.DataArray ()>
array(0.)
Coordinates:
    bin      int32 1
    nvars    int32 2

<xarray.DataArray ()>
array(0.)
Coordinates:
    bin      int32 2
    nvars    int32 2

<xarray.DataArray ()>
array(0.)
Coordinates:
    bin      int32 3
    nvars    int32 2

<xarray.DataArray ()>
array(0.)
Coordinates:
    bin      int32 4
    nvars    int32 2

<xarray.DataArray ()>
array(4.96733863e+09)
Coordinates:
    bin      int32 5
    nvars    int32 3

<xarray.DataArray ()>
array(0.)
Coordinates:
    bin      int32 6
    nvars    int32 2